In [50]:
import string
import networkx as nx
import matplotlib.pyplot as plt
import random

In [51]:
def euclidean_dist(a, b, coord):
    (x1, y1) = coord[a]
    (x2, y2) = coord[b]
    return ((x1-x2)**2+(y1-y2)**2)**0.5

def manhattan_dist(a, b, coord):
    (x1, y1) = coord[a]
    (x2, y2) = coord[b]
    return abs(x1 - x2) + abs(y1 - y2)

def non_informative(a,b):
    return 0

def ravel(listsoflists):
    return [item for elem in listsoflists for item in elem]

def node_neighbors(graph, node):
    return [exit for _, exit in graph.edges(node) if exit!=node]

In [52]:
def create_maze(seed=2, drawing=True):
    random.seed(seed)
    letters = [l for l in string.ascii_uppercase[:25]]
    checkboard = [letters[i:i+5] for i in range(0, len(letters), 5)]
    Graph = nx.Graph()
    for j, node in enumerate(letters):
        Graph.add_nodes_from(node)
        x, y = j // 5, j % 5
        x_min = max(0, x-1)
        x_max = min(4, x+1)+1
        y_min = max(0, y-1)
        y_max = min(4, y+1)+1
        adjacent_nodes = ravel([row[y_min:y_max] for row in checkboard[x_min:x_max]])
        exits = random.sample(adjacent_nodes, k=random.randint(1, 4))
        for exit in exits:
            if exit not in node_neighbors(Graph, node):
                Graph.add_edge(node, exit)
    spacing = [0.0, 0.2, 0.4, 0.6, 0.8]
    coordinates = [[x, y] for x in spacing \
                   for y in spacing]
    position  = {l:c for l,c in zip(letters, coordinates)}

    for node in Graph.nodes:
        for exit in node_neighbors(Graph, node):
            length = int(round(
                    euclidean_dist(
                        node, exit, position)*10,0))
            Graph.add_edge(node,exit,weight=length)

    if drawing:
        draw_params = {'with_labels':True, 
                       'node_color':'skyblue',
                       'node_size':700, 'width':2, 
                       'font_size':14}
        nx.draw(Graph, position, **draw_params)
        labels = nx.get_edge_attributes(Graph,'weight')
        nx.draw_networkx_edge_labels(Graph, position, 
                                     edge_labels=labels)
        plt.show()
    
    return Graph, position

In [53]:
graph, coordinates = create_maze(seed=7)

In [54]:
def graph_weight(graph, a, b):
    return graph.edges[(a, b)]['weight']

In [55]:
def reconstruct_path(connections, start, goal):
    if goal in connections:
        current = goal
        path = [current]
        while current != start:
            current = connections[current]
            path.append(current)
        return path[::-1]

def compute_path_dist(path, graph):
    if path:
        run = 0
        for step in range(len(path)-1):
            A = path[step]
            B = path[step+1]
            run += graph_weight(graph, A, B)
        return run
    else:
        return 0

In [56]:
graph, coordinates = create_maze(seed=30)
start = 'A'
goal  = 'Y'
scoring = manhattan_dist

In [57]:
# Best-first search
path = {}
open_list = set(graph.nodes())
closed_list = {start: manhattan_dist(start, goal, 
                                     coordinates)}

while open_list:
    
    candidates = open_list&closed_list.keys()
    if len(candidates)==0:
        print ("Cannot find a way to the goal %s" % goal)
        break
    frontier = [(closed_list[node], 
                 node) for node in candidates]
    score, min_node =sorted(frontier)[0]

    if min_node==goal:
        print ("Arrived at final vertex %s" % goal)
        print ('Unvisited vertices: %i' % (len(
                    open_list)-1))
        break
    else:
        print("Processing vertex %s, " % min_node, end="")

    open_list = open_list.difference(min_node)
    neighbors = node_neighbors(graph, min_node)
    to_be_visited = list(neighbors-closed_list.keys())
    
    if len(to_be_visited) == 0:
        print ("found no exit, retracing to %s" 
               % path[min_node])
    else:
        print ("discovered %s" % str(to_be_visited))

    for node in neighbors:
        if node not in closed_list:
            closed_list[node] = scoring(node, goal, 
                                        coordinates)
            path[node] = min_node

print ('\nBest path is:', reconstruct_path(
        path, start, goal))
print ('Length of path: %i' % compute_path_dist(
        reconstruct_path(path, start, goal), graph))

In [58]:
# A* 
open_list = set(graph.nodes())
closed_list = {start: manhattan_dist(
        start, goal, coordinates)}
visited = {start: 0}
path = {}

while open_list:
    
    candidates = open_list&closed_list.keys()
    if len(candidates)==0:
        print ("Cannot find a way to the goal %s" % goal)
        break
    frontier = [(closed_list[node], 
                 node) for node in candidates]
    score, min_node =sorted(frontier)[0]

    if min_node==goal:
        print ("Arrived at final vertex %s" % goal)
        print ('Unvisited vertices: %i' % (len(
                open_list)-1))
        break
    else:
        print("Processing vertex %s, " % min_node, end="")

    open_list = open_list.difference(min_node)
    current_weight = visited[min_node]
    neighbors = node_neighbors(graph, min_node)
    to_be_visited = list(neighbors-visited.keys())
        
    for node in neighbors:
        new_weight = current_weight + graph_weight(
                     graph, min_node, node)
        if node not in visited or \
        new_weight < visited[node]:
            visited[node] = new_weight
            closed_list[node] = manhattan_dist(node, goal,
                        coordinates) + new_weight
            path[node] = min_node
    
    if to_be_visited:
        print ("discovered %s" % to_be_visited)
    else:
        print ("getting back to open list")

print ('\nBest path is:', reconstruct_path(
        path, start, goal))
print ('Length of path: %i' % compute_path_dist(
        reconstruct_path(path, start, goal), graph))